In [47]:
import pandas as pd
from zipfile import ZipFile
from StringIO import StringIO
import urllib2
import numpy as np
import datetime
from scipy.stats import chisquare

In [1]:
ride_frames = []

months = ['04']#['01', '02', '03', '04', '05', '06']
for month in months:
    url = "https://s3.amazonaws.com/tripdata/2015%s-citibike-tripdata.zip" % month
    r = urllib2.urlopen(url).read()
    f = ZipFile(StringIO(r))
    csv = f.open("2015%s-citibike-tripdata.csv" % month)
    frame = pd.read_csv(csv, parse_dates=['starttime'])
    ride_frames.append(frame)
    
rides = pd.concat(ride_frames, ignore_index=True)

In [29]:
rides['day'] = rides['starttime'].dt.dayofweek
rides['date'] = rides['starttime'].dt.dayofyear
weekends = rides[(rides.day == 5) | (rides.day == 6)] 
weekdays = rides[(rides.day != 5) & (rides.day != 6)]

In [35]:
num_weekends = len(np.unique(weekends['date']))
num_weekdays = len(np.unique(weekdays['date']))

In [36]:
custWknds = float(len(weekends[weekends.usertype == 'Customer']))/num_weekends
subWknds = float(len(weekends[weekends.usertype == 'Subscriber']))/num_weekends
custWkday = float(len(weekdays[weekdays.usertype == 'Customer']))/num_weekdays
subWkday = float(len(weekdays[weekdays.usertype == 'Subscriber']))/num_weekdays

In [37]:
print 'Weekend customer rides = %.2f' % (custWknds)
print 'Weekend subscriber rides = %.2f' % (subWknds)
print "Weekend subscriber: weekend customer ratio = %.2f" % (subWknds/custWknds)
print 'Weekday customer rides = %.2f' % (custWkday)
print 'Weekday subscriber rides = %.2f' % (subWkday)
print "Weekday subscriber: weekday customer ratio = %.2f" % (subWkday/custWkday)

Weekend customer rides = 4315.88
Weekend subscriber rides = 15568.62
Weekend subscriber: weekend customer ratio = 3.61
Weekday customer rides = 1440.27
Weekday subscriber rides = 20983.09
Weekday subscriber: weekday customer ratio = 14.57


In [41]:
print '%% of Rides that are customers on weekends = %.2f' % (custWknds/(custWknds + subWknds))
print '%% of Rides that are customers on weekdays = %.2f' % (custWkday/(custWkday + subWkday))

% of Rides that are customers on weekends = 0.22
% of Rides that are customers on weekdays = 0.06


In [42]:
print custWknds + subWknds
print custWkday + subWkday

19884.5
22423.3636364


In [45]:
print custWknds + custWkday
print subWknds + subWkday

5756.14772727
36551.7159091


In [44]:
print subWknds + subWknds + custWknds + custWkday

36893.3977273


In [46]:
Ntot = 36893.3977273
expected = 5756.14772727*36551.71*22423.3636364*19884.5
sample_values = [[1440.27, 20983.09],[4315.88,15568.62]]
 
chisqstat= lambda N, values, expect : N*((values[0][0]*values[1][1]-values[0][1]*values[1][0])**2)/(expect)

print chisqstat(Ntot,  sample_values, expected)

1825.85275292


In [48]:
chisquare(sample_values)

(array([ 1436.57355561,   802.05509895]),
 array([  0.00000000e+000,   1.92861851e-176]))

In [53]:
chisquare([600, 2200], [600, 600])

(4266.666666666667, 0.0)

In [56]:
chisquare([1440.27, 4315.88], [1440.27, 19884.5*.06])

(8173.8224044691442, 0.0)

In [76]:
# Test whether the total customers on weekends is different than weekday
# The expected value for the weekday is the weekday count and the expected value for the weekend is
# The proportion of customers on weekdays times the total number of rides on weekends.
totalWknds = custWknds + subWknds
totalWkday = custWkday + subWkday
custWkdayPercent = custWkday/totalWkday
expected = [custWkday, totalWknds*custWkdayPercent]
sample = [custWkday, custWknds]
#sample = [custWkday, 1275]
chisquare(sample, expected)

(7229.5300960536615, 0.0)

In [72]:
chisquare([1440.27, 4315.88], [1440.27, 4315.88])

(0.0, 1.0)

In [69]:
print 19884.5*.06

1193.07
